In [1]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3MB 703kB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [2]:



# Upload the train file from your local drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/My\ Drive/Dr_AI


/content/drive/My Drive/Dr_AI


In [0]:
import pandas as pd
import torch 
from torch import nn


import numpy as np
from torch.utils.data import DataLoader
from torchtext.data import Field, Dataset, Example
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,hamming_loss

import pickle

In [0]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle
)

In [0]:
tokenizer.sequences_to_texts([[1,2,3]])

In [0]:

class DataFrameDataset(Dataset):
 """Class for using pandas DataFrames as a datasource"""
#  def __init__(self, fields, path='./test1.h5', filter_pred=None):
 def __init__(self, fields, examples, filter_pred=None): 
    """
     Create a dataset from a pandas dataframe of examples and Fields
     Arguments:
         examples pd.DataFrame: DataFrame of examples
         fields {str: Field}: The Fields to use in this tuple. The
             string is a field name, and the Field is the associated field.
         filter_pred (callable or None): use only exanples for which
             filter_pred(example) is true, or use all examples if None.
             Default is None
    """
#     examples = pd.read_hdf(path)
    self.examples = examples.apply(SeriesExample.fromSeries, args=(fields,), axis=1).tolist()
    if filter_pred is not None:
         self.examples = filter(filter_pred, self.examples)
    self.fields = dict(fields)
     # Unpack field tuples
    for n, f in list(self.fields.items()):
         if isinstance(n, tuple):
            self.fields.update(zip(n, f))
            del self.fields[n]

class SeriesExample(Example):
    """Class to convert a pandas Series to an Example"""

    @classmethod 
    def fromSeries(cls, data, fields):
        return cls.fromdict(data.to_dict(), fields)

    @classmethod
    def fromdict(cls, data, fields):
        ex = cls()
        fields = dict(fields)
#         print(fields.items())
        for key, field in fields.items():
#             print(key)
            if key not in data:
                 raise ValueError("Specified key {} was not found in "
         "the input data".format(key))
            if field is not None:
                setattr(ex, key, field.preprocess(data[key]))
#                 print(field.preprocess(data[key]))   
            else:
                 setattr(ex, key, data[key])
#         print(ex.input.pad_)        
        return ex

In [0]:
# LABEL = Field(sequential=True,use_vocab=False,batch_first=True,pad_token=[0]*4386)
# INPUT = Field(sequential=True,use_vocab=False,batch_first=True,pad_token=[0]*1364)
# LENGTH = Field(sequential=False,use_vocab=False,batch_first=True,pad_token=0)


LABEL = Field(sequential=True,use_vocab=False,batch_first=True,pad_token=[0]*2806)
INPUT = Field(sequential=True,use_vocab=False,batch_first=True,pad_token=[0]*1343)
LENGTH = Field(sequential=False,use_vocab=False,batch_first=True,pad_token=0)

In [0]:
fields = [('label',LABEL),('input',INPUT),('length',LENGTH)]

In [0]:
# final_df2 = pd.read_hdf('./test_ICD.h5')
final_df2 = pd.read_hdf('./m_test.h5')


In [0]:
final_df2.columns
final_df2 = final_df2.rename({'labels':'label', 'lengths':'length'},axis=1)

In [0]:
train, test = train_test_split(final_df2, test_size=0.1)
train, val = train_test_split(train, test_size=0.2)

In [0]:
train_ds = DataFrameDataset(fields, train)
val_ds = DataFrameDataset(fields, val)

In [0]:
test_ds = DataFrameDataset(fields=fields, examples=test)

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 


In [0]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [0]:
from torchtext.data import Iterator, BucketIterator

train_iter, val_iter, test_iter = BucketIterator.splits(
 (train_ds,val_ds,test_ds), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(BATCH_SIZE, BATCH_SIZE,BATCH_SIZE),
 device=device, # if you want to use the GPU, specify the GPU number here
 sort_key=lambda x: x.length, # the BucketIterator needs to be told what function it should use to group the data.
 sort_within_batch=True,
 repeat=False # we pass repeat=False because we want to wrap this Iterator layer.
)
# test_iter = Iterator(tst, batch_size=64, device=-1, sort=False, sort_within_batch=False, repeat=False)

In [0]:
# for batch in train_iter:
#     print(f'{batch.input[0]}')

In [0]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, num_emd, drop_prob=0.2, wv_file=None, tokenizer=None ):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.drop_prob = drop_prob

        self.embedding = nn.Embedding(embedding_dim=input_dim, num_embeddings=num_emd, padding_idx=0)
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
        self.wv_file =  wv_file
        self.tokenizer = tokenizer
        
    def forward(self, x,length):
        out = self.embedding(x)
#         print(f'before sum {out.shape}')

        out = out.sum(dim=-2)
        # print(f'before packing {out.shape} ')
        out = torch.nn.utils.rnn.pack_padded_sequence(out, lengths=length, batch_first=True, enforce_sorted=True)
        # print(f'after packing {out.data.shape}  {out.batch_sizes} {length} {out}')
           
        packed_output, h = self.gru(out)
        out, out_lengths = torch.nn.utils.rnn.pad_packed_sequence(packed_output)
#         print(f'after unpacking {out.shape} {out_lengths.shape}')
        out = out.transpose(0,1)
        out = self.fc(out)

        out=self.sigmoid(out)
        return out, h
    
    def init_weights( self, batch_size):
        
      if self.wv_file:
          print(self.wv_file)
          from gensim.models import KeyedVectors, Word2Vec

          wv = KeyedVectors.load(self.wv_file)
          word_indexer  = self.tokenizer.word_index
          
          wv_2 = {}

          for word in word_indexer:
              try:
                  wv_2[word_indexer[word]]= wv[word]
              except KeyError as k:
                  wv_2[word_indexer[word]] = np.zeros(wv.vector_size)

          wv_2
          x = [v for k, v in sorted(wv_2.items(), key=lambda item: item[0], reverse=False)]

          pad = np.zeros_like(x[0])
          x.insert(0,pad)
          y = torch.tensor(x, dtype=torch.float32)
          print(f'y  {y.shape}')
          device = self.embedding.weight.device
          self.embedding.weight =  torch.nn.Parameter(y)
          self.embedding = self.embedding.to(device)
          self.gru = nn.GRU(wv.vector_size, self.hidden_dim, self.n_layers, batch_first=True, dropout=self.drop_prob).to(device)

      self.init_hidden(batch_size)

          
    def init_hidden(self, batch_size ):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

In [0]:
# model = GRUNet(input_dim=1000, hidden_dim=2000, output_dim=4386, n_layers=2, num_emd=8200 )
  model = GRUNet(input_dim=1000, hidden_dim=2000, output_dim=2806, n_layers=2, num_emd=8200)


In [30]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 55,838,806 trainable parameters


In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(),weight_decay=0.01)

In [0]:
criterion = nn.BCELoss()
# criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def compute_loss(model, iterator, criterion):
    loss = 0
    
    for batch in iterator:
        text = batch.input
        text_lengths = batch.length

        predictions,h = model(text, text_lengths)
#         print(f'predictions.shape {predictions}')
#         print(f'\n\n labels \n{batch.label.dtype} \n\n predictions\n {predictions.dtype}')
        label = batch.label.to(dtype=torch.float32)
        loss += criterion(predictions, label).item()
    return loss/len(iterator)

In [0]:
k = 30

In [0]:
def ham_loss(model, iterator, k=60):
    acc = 0
    n_samples = 0
    recall = []
    
    for batch in iterator:
        text = batch.input
        text_lengths = batch.length

        predictions,h = model(text, text_lengths)
        predictions = (predictions.detach().cpu().numpy()).reshape(predictions.shape[0]*predictions.shape[1],-1)
        labels = np.array(batch.label.cpu()).reshape(batch.label.shape[0]*batch.label.shape[1],-1)
        
#         pred_seq = [np.where(x[0]==1)[0] for x in predictions]
        label_seq = [np.where(x==1)[0] for x in labels]
#         predictions = predictions.reshape(predictions.shape[0]*predictions.shape[1],-1)
        try :    
            for p,q in zip(label_seq,predictions):
                pred =np.sort(np.argsort(q)[-k:])
                acc += (np.intersect1d(pred,p)).size
                n_samples+=(p.size)   
                rec = (np.intersect1d(pred,p)).size/p.size
                recall.append(rec)
    #             print(acc)
    #             print(n_samples, len(p))
    #         acc += np.sum(np.not_equal(pred_seq, label_seq))
        except ZeroDivisionError as z:
            continue
        
    return(np.mean(recall))       

In [37]:
model

GRUNet(
  (embedding): Embedding(8200, 1000, padding_idx=0)
  (gru): GRU(1000, 2000, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=2000, out_features=2806, bias=True)
  (sigmoid): Sigmoid()
)

In [38]:
print(f'train {ham_loss(model=model,iterator=train_iter)} \n\n val {ham_loss(model=model,iterator=val_iter)}')
ham_loss(model=model,iterator=test_iter, k=100)

train 0.02292466404101867 

 val 0.02203229313942063


0.03859723546247425

In [0]:


def train(model, train_iterator, optimizer, criterion, val_iterator,epochs=100):
    
    metrics = []
    epoch_loss = 0
    val_loss = 0
    
    model.train()
    h = model.init_weights(batch_size=BATCH_SIZE)

    for epoch in range(epochs):  
        epoch_loss = 0
        for batch in train_iterator:

            optimizer.zero_grad()

            text = batch.input
            text_lengths = batch.length
            # print(text)
            predictions,h = model(text, text_lengths)
            # print(f'predictions.shape {predictions}')
    #         print(f'\n\n labels \n{batch.label.dtype} \n\n predictions\n {predictions.dtype}')
            label = batch.label.to(dtype=torch.float32)
            # print(f'{predictions.shape} {label.shape}')
            loss = criterion(predictions, label)
    #         acc = binary_accuracy(predictions, batch.label)

            loss.backward(retain_graph=True)
            val_loss+=compute_loss(model, val_iter,criterion)
            optimizer.step()

            epoch_loss += loss.item()
    #         epoch_acc += acc.item()
        print(f'epoch {epoch+1} epoch_loss {epoch_loss/len(train_iterator)} val_loss {val_loss} ')
        metrics.append(epoch_loss)
    return epoch_loss / len(train_iterator), np.mean(val_loss)

In [0]:
# %debug

In [40]:
model

GRUNet(
  (embedding): Embedding(8200, 1000, padding_idx=0)
  (gru): GRU(1000, 2000, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=2000, out_features=2806, bias=True)
  (sigmoid): Sigmoid()
)

In [0]:
config ={
    
    "embed_dim" : model.embedding.embedding_dim,
    "gru_hidden_size":model.gru.hidden_size,
    "gru_layers":  model.gru.num_layers,
    "avtivation":'sigmoid',
        
}


In [42]:
import numpy as np
K = np.arange(100,201,10)
K

array([100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200])

In [44]:
!mkdir all/2
!mkdir medicines/4/

mkdir: cannot create directory ‘all/2’: File exists


In [0]:
# folder ='all/4/'
folder ='medicines/4/'

In [0]:
import json, os
with open(os.path.join(folder,'config.json'), 'w') as fp:
    json.dump(config, fp, indent=1)

In [0]:
import os
from collections import defaultdict
from matplotlib import pyplot as plt    

loses_list = ['train_loss','val_loss',]
recall_list = ['train_recall', 'test_recall']    
losses = defaultdict(list)    
recalls = defaultdict(lambda: defaultdict(list))

for i in range(100):
    print('started_training')
    train_loss, val_loss = train(model=model, train_iterator=train_iter,
                                 optimizer=optimizer, criterion=criterion, val_iterator=val_iter, epochs=1)
    print('trained ', i)
    losses['train'].append(train_loss)
    losses['val'].append(val_loss)
    
    plt.style.use("ggplot")
    plt.figure()
    N = np.arange(1,len(losses['train'])+1)
    plt.plot(N,losses['train'], label='Train_loss')
    plt.plot(N,losses['val'], label='Val_loss')
    plt.legend()
    plt.xlabel("Epoch #")
    key ='loss'
    plt.ylabel(key)
    plt.title(f"Training {key} {i}")
    plt.savefig(os.path.join(folder,f'_{key}.png'))
    plt.close()  




    for k in K:
        recalls['train'][k].append(ham_loss(model=model,iterator=train_iter, k=k))
        recalls['test'][k].append(ham_loss(model=model,iterator=test_iter, k=k))

        plt.plot()
        plt.style.use("ggplot")
        plt.figure()
        N = np.arange(1,len(recalls['train'][k])+1)
        plt.plot(N,recalls['train'][k], label=f'Train_recall_{k}')
        plt.plot(N,recalls['test'][k], label=f'Test_recall_{k}')
        plt.legend()
        plt.xlabel("Epoch #")
        key ='recall'
        plt.ylabel(key)
        plt.title(f"Training {key} {i}")
        plt.savefig(os.path.join(folder,f'_{key}_{k}.png'))
        plt.close()  
    
    

started_training
epoch 1 epoch_loss 0.1680256108174453 val_loss 14.363899116061235 
trained  0
started_training
epoch 1 epoch_loss 0.11146084954207008 val_loss 10.557261847743858 
trained  1
started_training
epoch 1 epoch_loss 0.11267655060903446 val_loss 10.568049166547624 
trained  2
started_training
epoch 1 epoch_loss 0.11295321221287186 val_loss 10.618072965819588 
trained  3
started_training
epoch 1 epoch_loss 0.11453093779650894 val_loss 10.765679982931996 
trained  4
started_training
epoch 1 epoch_loss 0.11360115114901517 val_loss 10.649198083108976 
trained  5
started_training
epoch 1 epoch_loss 0.11473963409662247 val_loss 10.778784590332133 
trained  6
started_training
epoch 1 epoch_loss 0.11480067811302237 val_loss 10.768587931206351 
trained  7
started_training
epoch 1 epoch_loss 0.11608087019743146 val_loss 10.860725880453465 
trained  8
started_training
epoch 1 epoch_loss 0.11569054092507104 val_loss 10.857970436152662 
trained  9
started_training
epoch 1 epoch_loss 0.117

In [0]:
%debug

In [0]:
train_recall_125 = [ham_loss(model=model,iterator=train_iter, k=125)]
test_recall_125 = [ham_loss(model=model,iterator=test_iter, k=125)]
train_recall_100 = [ham_loss(model=model,iterator=train_iter, k=100)]
test_recall_100 = [ham_loss(model=model,iterator=test_iter,k=100)]
train_recall_150 = [ham_loss(model=model,iterator=train_iter, k=150)]
test_recall_150 = [ham_loss(model=model,iterator=test_iter,k=150)]

for i in range(10):
    train(model=model, train_iterator=train_iter, optimizer=optimizer, criterion=criterion,
            val_iterator=val_iter, epochs=10)
    train_recall_125.append(ham_loss(model=model,iterator=train_iter, k=125))
    test_recall_125.append(ham_loss(model=model,iterator=test_iter, k=125))
    train_recall_100.append(ham_loss(model=model,iterator=train_iter, k=100))
    test_recall_100.append(ham_loss(model=model,iterator=test_iter, k=100))
    train_recall_150.append(ham_loss(model=model,iterator=train_iter, k=150))
    test_recall_150.append(ham_loss(model=model,iterator=test_iter, k=150))

In [0]:
test_recall_100

In [0]:
%debug

In [0]:
torch.save(model.state_dict(), 'model3.pt')


In [0]:
model.load_state_dict(torch.load('model3.pt'))


In [0]:
val_loss=evaluate(model, val_iter,criterion)


In [0]:
val_loss

In [0]:
len(train_iter)

In [0]:
%debug

In [0]:

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [0]:
def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=5, model_type="GRU"):
    
    # Setting common hyperparameters
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = input_dim
    n_layers = 1
    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.clock()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()
            
            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.clock()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

def evaluate(model, test_x, test_y, label_scalers):
    model.eval()
    outputs = []
    targets = []
    start_time = time.clock()
    for i in test_x.keys():
        inp = torch.from_numpy(np.array(test_x[i]))
        labs = torch.from_numpy(np.array(test_y[i]))
        h = model.init_hidden(inp.shape[0])
        out, h = model(inp.to(device).float(), h)
        outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
        targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
    print("Evaluation Time: {}".format(str(time.clock()-start_time)))
    sMAPE = 0
    for i in range(len(outputs)):
        sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
    print("sMAPE: {}%".format(sMAPE*100))
    return outputs, targets, sMAPE

In [0]:
new_array = np.array([1.2, 3.4, 5.6])

In [0]:
train_ds[0].label.shape

In [0]:
x = BucketIterator(train_ds, batch_size=4, sort_key='length')
# for y in x:
#     y

In [0]:
x.create_batches

In [0]:
%debug

In [0]:
train_ds= DataFrameDataset(fields= fields, examples=train.iloc[:1])

In [0]:
y = BucketIterator(train_ds, batch_size=2,sort_key='length')

In [0]:
for x in y:
    print(x)

In [0]:
# train
%debug

In [0]:
x =torch.randn((3,8199))
model(x,3)

In [0]:
10 epochs : 0.55(100) , 0.4(60)

In [0]:
import os
from pathlib import Path
with open('../Gensim/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
wv_folder ='../Gensim/SG_data_2/word2vec/'

files = os.listdir(wv_folder)



import torch.optim as optim



files = [x for x in filter(lambda x:x.split("_")[0] == 'all', files)]
files  = [x for x in filter(lambda x:x.split("_")[2] in ['500'] and x.split("_")[3].split('.')[1] == 'wv', files)]

file_paths = [ os.path.join(wv_folder, file) for file in files]

K = np.arange(100,201,10)






import os
from collections import defaultdict
from matplotlib import pyplot as plt    


for file,file_path in zip(files,file_paths[0:]):

  folder = Path((os.path.join("SG2",(file).split('_')[0]+'/'+ (file).split('_')[3].split('.')[0]+'/'+file.split('_')[2])))
  folder.mkdir(parents=True,exist_ok=True)
  print(folder)
  print(f'file_path {file_path}')
  model = GRUNet(input_dim=100, hidden_dim=2000, output_dim=4386, n_layers=2, num_emd=8120, wv_file=file_path, tokenizer=tokenizer )
  model.init_weights(batch_size= BATCH_SIZE)
  model = model.to(device)
  optimizer = optim.Adam(model.parameters(), weight_decay=0.001)

  criterion = nn.BCELoss(reduction='mean')
  
  
  criterion = criterion.to(device)
  
  # print(f'1 {device} {model.embedding.weight.device}')
  config ={
      
      "embed_dim" : model.embedding.embedding_dim,
      "gru_hidden_size":model.gru.hidden_size,
      "gru_layers":  model.gru.num_layers,
      "avtivation":'sigmoid',
      "wv": f'{file}',
          
  }

  import json
  with open(os.path.join(folder, 'config.json'), 'w') as fp:
      json.dump(config, fp, indent=1)


  loses_list = ['train_loss','val_loss',]
  recall_list = ['train_recall', 'test_recall']    
  losses = defaultdict(list)    
  recalls = defaultdict(lambda: defaultdict(list))

  for i in range(30):
      print('started_training')
      # print(f'2 {device} {model.embedding.weight.device}')
      train_loss, val_loss = train(model=model, train_iterator=train_iter,
                                  optimizer=optimizer, criterion=criterion, val_iterator=val_iter, epochs=1)
      print('trained ', i)
      losses['train'].append(train_loss)
      losses['val'].append(val_loss)
      
      plt.style.use("ggplot")
      plt.figure()
      N = np.arange(1,len(losses['train'])+1)
      plt.plot(N,losses['train'], label='Train_loss')
      plt.plot(N,losses['val'], label='Val_loss')
      plt.legend()
      plt.xlabel("Epoch #")
      key ='loss'
      plt.ylabel(key)
      plt.title(f"Training {key} {i}")
      plt.savefig(os.path.join(folder,f'_{key}.png'))
      plt.close()  


      

      for k in K:
          recalls['train'][k].append(ham_loss(model=model,iterator=train_iter, k=k))
          recalls['test'][k].append(ham_loss(model=model,iterator=test_iter, k=k))

          plt.plot()
          plt.style.use("ggplot")
          plt.figure()
          N = np.arange(1,len(recalls['train'][k])+1)
          plt.plot(N,recalls['train'][k], label=f'Train_recall_{k}')
          plt.plot(N,recalls['test'][k], label=f'Test_recall_{k}')
          plt.legend()
          plt.xlabel("Epoch #")
          key ='recall'
          plt.ylabel(key)
          plt.title(f"Training {key} {i}")
          plt.savefig(os.path.join(folder,f'{key}_{k}.png'))
          plt.close()  
      


SG2/all/5/500
all_data_500_5.wv
../Gensim/SG_data_2/word2vec/all_data_500_5.wv


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


y  torch.Size([8199, 500])
started_training
epoch 1 epoch_loss 0.08422979041934013 val_loss 8.9911577174948
trained  0
started_training
epoch 1 epoch_loss 0.06326153764829916 val_loss 8.018481517210603
trained  1
started_training
epoch 1 epoch_loss 0.06450602973208708 val_loss 8.117772364650259
trained  2
started_training
epoch 1 epoch_loss 0.06598878818399766 val_loss 8.247509508647699
trained  3
started_training
epoch 1 epoch_loss 0.06730359897894019 val_loss 8.33284658515318
trained  4
started_training
epoch 1 epoch_loss 0.06790340700570274 val_loss 8.363020767542446
trained  5
started_training
epoch 1 epoch_loss 0.06896878660601728 val_loss 8.468369912017476
trained  6
started_training
epoch 1 epoch_loss 0.06924150739522542 val_loss 8.471003014763648
trained  7
started_training
epoch 1 epoch_loss 0.07084793102215318 val_loss 8.664397790858693
trained  8
started_training
epoch 1 epoch_loss 0.07004529921447529 val_loss 8.524045661091805
trained  9
started_training
epoch 1 epoch_loss 

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


y  torch.Size([8199, 500])
started_training
epoch 1 epoch_loss 0.08253124579787255 val_loss 8.832640887153413
trained  0
started_training
epoch 1 epoch_loss 0.06296422043267419 val_loss 7.975540513680741
trained  1
started_training
epoch 1 epoch_loss 0.064579789384323 val_loss 8.106155736371877
trained  2
started_training
epoch 1 epoch_loss 0.06558058283784811 val_loss 8.188554568216205
trained  3
started_training
epoch 1 epoch_loss 0.06704350398743854 val_loss 8.317197033119468
trained  4
started_training
epoch 1 epoch_loss 0.0671622168053599 val_loss 8.320239981297743
trained  5
started_training
epoch 1 epoch_loss 0.06905111512717078 val_loss 8.458048848604612
trained  6
started_training
epoch 1 epoch_loss 0.06919546583119561 val_loss 8.501973955130037
trained  7
started_training
epoch 1 epoch_loss 0.06962373655508547 val_loss 8.490980000150476
trained  8
started_training
epoch 1 epoch_loss 0.06959048566572806 val_loss 8.501488050276583
trained  9
started_training
epoch 1 epoch_loss 

In [0]:
pwd

In [0]:
model.embedding.weight.shape

torch.Size([5394, 100])